# ETL Project
## Emaan Qillawala
## DS 2002 4/6/24

##### In this project, I extract data from a CSV, API, and SQL database and add it to a SQL database containing 5 relational tables about college men's basketball. At the end, I query into the data to extract various information. All of the code is conducted in Python and mySQL.

#### Prepare the SQL connection

In [59]:
# import all libraries

# libraries for using pandas and mySQL through Python
import pandas as pd
import numpy as np
import os
import pymysql
import warnings
import mysql.connector
warnings.filterwarnings('ignore')

# libraries for using API call and returning JSON
import os
import json
import pprint
import requests
import requests.exceptions

In [60]:
# create database in mySQL

# connect to mySQL server
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Arrakis1!Paul"
)

mycursor = mydb.cursor()

# create database
mycursor.execute("CREATE DATABASE db_ncaa_mens_bball")

In [61]:
# Code to connect to local mySQL

# local connection information
host_name = 'localhost'
# host_ip = 127.0.0.1
port = '3306'

user_id = 'root'
pwd = 'Arrakis1!Paul'
db_name = 'db_ncaa_mens_bball'

# these two lines of code will be used repeatedly to connect to mySQL and create queries
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

## Part 1: Create 5 relational tables in a SQL database with primary and foreign keys

In [62]:
# Create Table 1: team_info

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the table 
try:

    cursor.execute("CREATE TABLE team_info (team_id INT AUTO_INCREMENT PRIMARY KEY, school_name VARCHAR(50), conference VARCHAR(50), games_played INT, games_won INT, season_year INT, postseason VARCHAR(50));")

    cursor.close()

except:
    print ("Error: unable to create table")

# close connection
conn.close()

In [63]:
# Create Table 2: postseason

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the table

try:

    cursor.execute("CREATE TABLE postseason (postseason_id INT AUTO_INCREMENT PRIMARY KEY, champion VARCHAR(50), season_year INT, final_score VARCHAR(50), game_site VARCHAR(50),coach VARCHAR(50), runner_up VARCHAR(50));")

    cursor.close()

except:
    print ("Error: unable to create table")

# close connection
conn.close()

In [64]:
# Create Table 3: skills_offense

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the table
try:

    cursor.execute("CREATE TABLE skills_offense (skills_offense_id INT AUTO_INCREMENT PRIMARY KEY, team_name VARCHAR(50), conference VARCHAR(50), adjusted_offensive_rate INT, offensive_rebounds INT, freethrow_rate INT, effective_field_goals INT, season_year INT);")

    cursor.close()

except:
    print ("Error: unable to create table")

# close connection
conn.close()

In [65]:
# Create Table 4: skills_defense

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the table
try:

    cursor.execute("CREATE TABLE skills_defense (skills_defense_id INT AUTO_INCREMENT PRIMARY KEY, school_name VARCHAR(50), conference VARCHAR(50), adjusted_defensive_rate INT, defensive_rebounds INT, freethrow_rate_allowed INT, effective_field_goals_allowed INT, year_season INT);")

    cursor.close()

except:
    print ("Error: unable to create table")

# close connection
conn.close()

In [66]:
 # Create Table 5: game_details

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the table
try:

    cursor.execute("CREATE TABLE game_details (game_details_id INT AUTO_INCREMENT PRIMARY KEY, venue_city VARCHAR(50), venue_state VARCHAR(50), venue_address VARCHAR(50), venue_name VARCHAR(50), season_year INT, home_team_name VARCHAR(50));")

except:
    print ("Error: unable to create table")

# close connection
conn.close()

## Part 2: Extract data from a CSV, API, and SQL database and insert this data into the SQL tables

#### Data Source 1 -> CSV file from Kaggle

##### The CSV file is on Github for reference.

In [67]:
# read in data
kaggle_bball_data = pd.read_csv('cbb.csv', sep = ',')
print(kaggle_bball_data.head())

             TEAM CONF   G   W  ADJOE  ADJDE  BARTHAG  EFG_O  EFG_D   TOR  \
0  North Carolina  ACC  40  33  123.3   94.9   0.9531   52.6   48.1  15.4   
1       Wisconsin  B10  40  36  129.1   93.6   0.9758   54.8   47.7  12.4   
2        Michigan  B10  40  33  114.4   90.4   0.9375   53.9   47.7  14.0   
3      Texas Tech  B12  38  31  115.2   85.2   0.9696   53.5   43.0  17.7   
4         Gonzaga  WCC  39  37  117.8   86.3   0.9728   56.6   41.1  16.2   

   ...  FTRD  2P_O  2P_D  3P_O  3P_D  ADJ_T   WAB  POSTSEASON  SEED  YEAR  
0  ...  30.4  53.9  44.6  32.7  36.2   71.7   8.6         2ND   1.0  2016  
1  ...  22.4  54.8  44.7  36.5  37.5   59.3  11.3         2ND   1.0  2015  
2  ...  30.0  54.7  46.8  35.2  33.2   65.9   6.9         2ND   3.0  2018  
3  ...  36.6  52.8  41.9  36.5  29.7   67.5   7.0         2ND   3.0  2019  
4  ...  26.9  56.3  40.0  38.2  29.0   71.5   7.7         2ND   1.0  2017  

[5 rows x 24 columns]


In [68]:
# Extract only relevant columns
kaggle_bball_sql_data = kaggle_bball_data[['TEAM','G', 'W', 'ADJOE', 'DRB', 'FTRD', 'FTR', 'EFG_O', 'ADJDE', 'CONF', 'WAB', 'ORB', '2P_O', 'EFG_D', 'YEAR', 'POSTSEASON']]
print(kaggle_bball_sql_data.head())
# drop NA observations
kaggle_bball_sql_data = kaggle_bball_sql_data.dropna()
# confirm data has been cleaned
kaggle_bball_sql_data.info()

             TEAM   G   W  ADJOE   DRB  FTRD   FTR  EFG_O  ADJDE CONF   WAB  \
0  North Carolina  40  33  123.3  30.0  30.4  32.3   52.6   94.9  ACC   8.6   
1       Wisconsin  40  36  129.1  23.7  22.4  36.2   54.8   93.6  B10  11.3   
2        Michigan  40  33  114.4  24.9  30.0  30.7   53.9   90.4  B10   6.9   
3      Texas Tech  38  31  115.2  28.7  36.6  32.9   53.5   85.2  B12   7.0   
4         Gonzaga  39  37  117.8  26.2  26.9  39.0   56.6   86.3  WCC   7.7   

    ORB  2P_O  EFG_D  YEAR POSTSEASON  
0  40.7  53.9   48.1  2016        2ND  
1  32.1  54.8   47.7  2015        2ND  
2  25.5  54.7   47.7  2018        2ND  
3  27.4  52.8   43.0  2019        2ND  
4  30.0  56.3   41.1  2017        2ND  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 680 entries, 0 to 3227
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM        680 non-null    object 
 1   G           680 non-null    int64  
 2   W         

In [69]:
# Insert data into team_info table

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# insert the relevant data
for index, row in kaggle_bball_sql_data.iterrows():
     cursor.execute("INSERT INTO team_info (school_name,conference,games_played,games_won,season_year,postseason) VALUES (%s,%s,%s,%s,%s,%s)", (row.TEAM, row.CONF,row.G,row.W,row.YEAR,row.POSTSEASON))

# confirm data added by querying
conn.commit()

try:
        cursor.execute('SELECT * FROM team_info;')

        for row in cursor.fetchall():
                print(row)
        cursor.close()

except:
        print ("Error: unable to fetch data")

# close connection
conn.close()

(1, 'North Carolina', 'ACC', 40, 33, 2016, '2ND')
(2, 'Wisconsin', 'B10', 40, 36, 2015, '2ND')
(3, 'Michigan', 'B10', 40, 33, 2018, '2ND')
(4, 'Texas Tech', 'B12', 38, 31, 2019, '2ND')
(5, 'Gonzaga', 'WCC', 39, 37, 2017, '2ND')
(6, 'Kentucky', 'SEC', 40, 29, 2014, '2ND')
(7, 'Michigan', 'B10', 38, 30, 2013, '2ND')
(8, 'Duke', 'ACC', 39, 35, 2015, 'Champions')
(9, 'Virginia', 'ACC', 38, 35, 2019, 'Champions')
(10, 'North Carolina', 'ACC', 39, 33, 2017, 'Champions')
(11, 'Villanova', 'BE', 40, 35, 2016, 'Champions')
(12, 'Villanova', 'BE', 40, 36, 2018, 'Champions')
(13, 'Connecticut', 'Amer', 40, 32, 2014, 'Champions')
(14, 'Louisville', 'BE', 40, 35, 2013, 'Champions')
(15, 'Louisville', 'ACC', 36, 27, 2015, 'E8')
(16, 'Notre Dame', 'ACC', 38, 32, 2015, 'E8')
(17, 'Notre Dame', 'ACC', 36, 24, 2016, 'E8')
(18, 'Virginia', 'ACC', 37, 29, 2016, 'E8')
(19, 'Duke', 'ACC', 37, 29, 2018, 'E8')
(20, 'Florida St.', 'ACC', 35, 23, 2018, 'E8')
(21, 'Duke', 'ACC', 38, 32, 2019, 'E8')
(22, 'Purdue'

In [70]:
# Insert data into skills_defense table

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# insert the relevant data
for index, row in kaggle_bball_sql_data.iterrows():
     cursor.execute("INSERT INTO skills_defense (school_name,conference,adjusted_defensive_rate,defensive_rebounds,freethrow_rate_allowed,effective_field_goals_allowed, year_season) VALUES (%s,%s,%s,%s,%s,%s,%s)", (row.TEAM, row.CONF,row.ADJDE,row.DRB,row.FTRD,row.EFG_D,row.YEAR))

# confirm data added by querying
conn.commit()

try: 
        cursor.execute('SELECT * FROM skills_defense;')

        for row in cursor.fetchall():
                print(row)

        cursor.close()

except:
       print ("Error: unable to fetch data")

# close connection
conn.close()

(1, 'North Carolina', 'ACC', 95, 30, 30, 48, 2016)
(2, 'Wisconsin', 'B10', 94, 24, 22, 48, 2015)
(3, 'Michigan', 'B10', 90, 25, 30, 48, 2018)
(4, 'Texas Tech', 'B12', 85, 29, 37, 43, 2019)
(5, 'Gonzaga', 'WCC', 86, 26, 27, 41, 2017)
(6, 'Kentucky', 'SEC', 96, 30, 37, 46, 2014)
(7, 'Michigan', 'B10', 94, 29, 23, 48, 2013)
(8, 'Duke', 'ACC', 91, 30, 24, 46, 2015)
(9, 'Virginia', 'ACC', 90, 25, 26, 45, 2019)
(10, 'North Carolina', 'ACC', 92, 25, 32, 48, 2017)
(11, 'Villanova', 'BE', 91, 29, 30, 47, 2016)
(12, 'Villanova', 'BE', 94, 27, 27, 48, 2018)
(13, 'Connecticut', 'Amer', 91, 33, 36, 45, 2014)
(14, 'Louisville', 'BE', 84, 33, 35, 45, 2013)
(15, 'Louisville', 'ACC', 87, 31, 33, 44, 2015)
(16, 'Notre Dame', 'ACC', 99, 32, 24, 48, 2015)
(17, 'Notre Dame', 'ACC', 103, 32, 26, 50, 2016)
(18, 'Virginia', 'ACC', 91, 25, 33, 48, 2016)
(19, 'Duke', 'ACC', 93, 30, 22, 46, 2018)
(20, 'Florida St.', 'ACC', 96, 30, 32, 48, 2018)
(21, 'Duke', 'ACC', 89, 30, 24, 45, 2019)
(22, 'Purdue', 'B10', 94, 

In [71]:
# Insert data into skills_offense table

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# insert the relevant data
for index, row in kaggle_bball_sql_data.iterrows():
     cursor.execute("INSERT INTO skills_offense (team_name,conference,adjusted_offensive_rate,offensive_rebounds,freethrow_rate,effective_field_goals,season_year) VALUES (%s,%s,%s,%s,%s,%s,%s)", (row.TEAM, row.CONF,row.ADJOE,row.ORB,row.FTR,row.EFG_O,row.YEAR))
     
# confirm data added by querying
conn.commit()

try:
        cursor.execute('SELECT * FROM skills_offense;')

        for row in cursor.fetchall():
                print(row)
        cursor.close()
       
except:
     print ("Error: unable to fetch data")

# close connection
conn.close()

(1, 'North Carolina', 'ACC', 123, 41, 32, 53, 2016)
(2, 'Wisconsin', 'B10', 129, 32, 36, 55, 2015)
(3, 'Michigan', 'B10', 114, 26, 31, 54, 2018)
(4, 'Texas Tech', 'B12', 115, 27, 33, 54, 2019)
(5, 'Gonzaga', 'WCC', 118, 30, 39, 57, 2017)
(6, 'Kentucky', 'SEC', 117, 42, 52, 50, 2014)
(7, 'Michigan', 'B10', 122, 32, 28, 55, 2013)
(8, 'Duke', 'ACC', 125, 36, 40, 57, 2015)
(9, 'Virginia', 'ACC', 123, 30, 29, 55, 2019)
(10, 'North Carolina', 'ACC', 121, 41, 34, 52, 2017)
(11, 'Villanova', 'BE', 123, 28, 34, 56, 2016)
(12, 'Villanova', 'BE', 128, 30, 29, 60, 2018)
(13, 'Connecticut', 'Amer', 112, 30, 38, 52, 2014)
(14, 'Louisville', 'BE', 116, 38, 40, 51, 2013)
(15, 'Louisville', 'ACC', 109, 35, 39, 48, 2015)
(16, 'Notre Dame', 'ACC', 125, 28, 37, 58, 2015)
(17, 'Notre Dame', 'ACC', 118, 33, 33, 54, 2016)
(18, 'Virginia', 'ACC', 120, 30, 32, 55, 2016)
(19, 'Duke', 'ACC', 122, 39, 35, 56, 2018)
(20, 'Florida St.', 'ACC', 113, 32, 37, 53, 2018)
(21, 'Duke', 'ACC', 119, 36, 33, 54, 2019)
(22, '

#### Data source 2 -> SQL database from Google Cloud

##### For this data source, the SQL file has already been converted to a SQL database and been placed into a table for easy use and extraction. See SQL file in Github for code details.

In [72]:
# information to connect to the SQL data source
host_name = 'localhost'
# host_ip = 127.0.0.1
port = '3306'

user_id = 'root'
pwd = 'Arrakis1!Paul'
db_name = 'SQL_file_data_source'

# connect to mySQL
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

In [73]:
# convert SQL table to dataframe
SQL_df = pd.read_sql("SELECT * FROM ncaa_mens_bball;", conn)

conn.close()
SQL_df.head()

,game_id,season,scheduled_date,attendance,lead_changes,times_tied,periods,venue_id,venue_city,venue_state,...,opp_points,opp_fast_break_pts,opp_second_chance_pts,opp_team_turnovers,opp_points_off_turnovers,opp_team_rebounds,opp_flagrant_fouls,opp_player_tech_fouls,opp_team_tech_fouls,opp_coach_tech_fouls
0,4069f80e-04f0-4f69-a563-86014bbe95a0,2015,12/3/15,1281.0,6.0,6.0,2,68e1c556-3a3d-4806-ba8f-c9407831cb71,San Diego,CA,...,56,0.0,3.0,0,10.0,4,0,0,0,0
1,7160a0e0-bbc3-46ad-afc6-e4e6b5b90a51,2015,1/17/16,2205.0,0.0,0.0,2,ee6551cc-e4b5-4e08-906e-6184b5f6ab71,Stockton,CA,...,30,0.0,0.0,1,5.0,4,0,0,1,0
2,320ccf7a-8a32-4ce6-a561-10687985c6a6,2015,12/22/15,1236.0,2.0,0.0,2,7ba1442a-fd2a-463f-82c0-c5ac49a7de0d,San Jose,CA,...,66,2.0,8.0,1,6.0,1,0,0,0,0
3,4069f80e-04f0-4f69-a563-86014bbe95a0,2015,12/3/15,1281.0,6.0,6.0,2,68e1c556-3a3d-4806-ba8f-c9407831cb71,San Diego,CA,...,71,0.0,17.0,0,31.0,7,0,0,0,0
4,7160a0e0-bbc3-46ad-afc6-e4e6b5b90a51,2015,1/17/16,2205.0,0.0,0.0,2,ee6551cc-e4b5-4e08-906e-6184b5f6ab71,Stockton,CA,...,88,21.0,16.0,1,30.0,3,0,0,0,0


In [74]:
# Extract only relevant columns

SQL_bball_data = SQL_df[['venue_city', 'venue_state', 'venue_address', 'season', 'venue_name', 'name']]
print(SQL_bball_data.head())
# Drop NA observations
SQL_bball_data = SQL_bball_data.dropna()
# Confirm that dataframe has been cleaned
SQL_bball_data.info()

  venue_city venue_state         venue_address  season             venue_name  \
0  San Diego          CA      5998 Alcala Park    2015   Jenny Craig Pavilion   
1   Stockton          CA  1178 Larry Heller Dr    2015  Alex G. Spanos Center   
2   San Jose          CA  290 South 7th Street    2015     Event Center Arena   
3  San Diego          CA      5998 Alcala Park    2015   Jenny Craig Pavilion   
4   Stockton          CA  1178 Larry Heller Dr    2015  Alex G. Spanos Center   

       name  
0   Toreros  
1    Tigers  
2  Spartans  
3    Flames  
4    Flames  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   venue_city     480 non-null    object
 1   venue_state    480 non-null    object
 2   venue_address  480 non-null    object
 3   season         480 non-null    int64 
 4   venue_name     480 non-null    object
 5   name           480 

In [75]:
# Insert data into game_details table

# information to connect back to original mySQL database
host_name = 'localhost'
# host_ip = 127.0.0.1
port = '3306'

user_id = 'root'
pwd = 'Arrakis1!Paul'
db_name = 'db_ncaa_mens_bball'

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# insert the relevant data
for index, row in SQL_bball_data.iterrows():
     cursor.execute("INSERT INTO game_details (venue_city,venue_state,venue_address,venue_name,season_year,home_team_name) VALUES (%s,%s,%s,%s,%s,%s)", (row.venue_city, row.venue_state, row.venue_address, row.venue_name, row.season, row.name))

# confirm data added by querying
conn.commit()

try:

        cursor.execute('SELECT * FROM game_details;')

        for row in cursor.fetchall():
                print(row)
        cursor.close()

except:
        print ("Error: unable to fetch data")

# close connection
conn.close()

(1, 'San Diego', 'CA', '5998 Alcala Park', 'Jenny Craig Pavilion', 2015, '0')
(2, 'Stockton', 'CA', '1178 Larry Heller Dr', 'Alex G. Spanos Center', 2015, '1')
(3, 'San Jose', 'CA', '290 South 7th Street', 'Event Center Arena', 2015, '2')
(4, 'San Diego', 'CA', '5998 Alcala Park', 'Jenny Craig Pavilion', 2015, '3')
(5, 'Stockton', 'CA', '1178 Larry Heller Dr', 'Alex G. Spanos Center', 2015, '4')
(6, 'San Jose', 'CA', '290 South 7th Street', 'Event Center Arena', 2015, '5')
(7, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '25')
(8, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '26')
(9, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '27')
(10, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '28')
(11, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '29')
(12, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '30')
(13, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '31')
(14, 'Tampa', 'FL', 'Elm Drive', 'USF Sun Dome', 2014, '32')
(15, 'Tampa', 'FL', 'Elm Drive', 'USF

#### Data source 3 -> API call from a Github page

##### Details on the API call and similar codes are on the Github page.

In [76]:
# define API function to call JSON

def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()
# include error statements for when using the function
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"

    return result

# API data source URL
valid_url = "https://ncaa-api.henrygd.me/history/basketball-men/d1"
# define returning data as specific data type
response_type = ['json', 'dataframe']

# return API data as JSON
json_string = get_api_response(valid_url, response_type[0])
print(json_string)

{
    "data": [
        {
            "Champion (Record)": "UConn (31-8)",
            "Coach": "Dan Hurley",
            "Runner-Up": "San Diego State",
            "Score": "76-59",
            "Site": "Houston, Tex.",
            "Year": "2023",
            "title game replays": "Watch the full game"
        },
        {
            "Champion (Record)": "Kansas (34-6)",
            "Coach": "Bill Self",
            "Runner-Up": "North Carolina",
            "Score": "72-69",
            "Site": "New Orleans, La.",
            "Year": "2022",
            "title game replays": "Watch the full game"
        },
        {
            "Champion (Record)": "Baylor (28-2)",
            "Coach": "Scott Drew",
            "Runner-Up": "Gonzaga",
            "Score": "86-70",
            "Site": "Indianapolis, Ind.",
            "Year": "2021",
            "title game replays": "Watch the full game"
        },
        {
            "Champion (Record)": "Canceled due to Covid-19",
            "

In [77]:
# read JSON as a dataframe
basketball = get_api_response(valid_url, response_type[0])
pd.read_json(basketball)

,data,page,pages,sport,title,updated
0,"{'Champion (Record)': 'UConn (31-8)', 'Coach':...",1,1,Men's Basketball,Championship History,
1,"{'Champion (Record)': 'Kansas (34-6)', 'Coach'...",1,1,Men's Basketball,Championship History,
2,"{'Champion (Record)': 'Baylor (28-2)', 'Coach'...",1,1,Men's Basketball,Championship History,
3,{'Champion (Record)': 'Canceled due to Covid-1...,1,1,Men's Basketball,Championship History,
4,"{'Champion (Record)': 'Virginia (35-3)', 'Coac...",1,1,Men's Basketball,Championship History,
...,...,...,...,...,...,...
80,"{'Champion (Record)': 'Wyoming (31-2)', 'Coach...",1,1,Men's Basketball,Championship History,
81,"{'Champion (Record)': 'Stanford (28-4)', 'Coac...",1,1,Men's Basketball,Championship History,
82,"{'Champion (Record)': 'Wisconsin (20-3)', 'Coa...",1,1,Men's Basketball,Championship History,
83,"{'Champion (Record)': 'Indiana (20-3)', 'Coach...",1,1,Men's Basketball,Championship History,


In [78]:
# convert JSON into dataframe
data = json.loads(basketball)
champ, coach, second, score, site, year, title = [],[],[],[],[],[],[]
for result in data['data']:
    champ.append(result[u'Champion (Record)'])
    coach.append(result[u'Coach'])
    second.append(result[u'Runner-Up'])
    score.append(result[u'Score'])
    site.append(result[u'Site'])
    year.append(result[u'Year'])
    title.append(result[u'title game replays'])
API_df = pd.DataFrame([champ, coach, second, score, site, year, title]).T

# add column titles to the df
API_df.columns = ['champions', 'coach', 'runner_up', 'final_score',
                     'game_site', 'season_year', 'title_game_replays']
print(API_df.head())

                  champions         coach        runner_up final_score  \
0              UConn (31-8)    Dan Hurley  San Diego State       76-59   
1             Kansas (34-6)     Bill Self   North Carolina       72-69   
2             Baylor (28-2)    Scott Drew          Gonzaga       86-70   
3  Canceled due to Covid-19            --               --          --   
4           Virginia (35-3)  Tony Bennett       Texas Tech  85-77 (OT)   

            game_site season_year   title_game_replays  
0       Houston, Tex.        2023  Watch the full game  
1    New Orleans, La.        2022  Watch the full game  
2  Indianapolis, Ind.        2021  Watch the full game  
3                  --        2020                   --  
4  Minneapolis, Minn.        2019  Watch the full game  


In [79]:
# Extract only relevant columns
API_data = API_df[['champions', 'coach', 'runner_up', 'final_score',
                     'game_site', 'season_year']]
print(API_data.head())
# Drop NA observations
API_data = API_data.dropna()
# Confirm that dataframe has been cleaned
API_data.info()

                  champions         coach        runner_up final_score  \
0              UConn (31-8)    Dan Hurley  San Diego State       76-59   
1             Kansas (34-6)     Bill Self   North Carolina       72-69   
2             Baylor (28-2)    Scott Drew          Gonzaga       86-70   
3  Canceled due to Covid-19            --               --          --   
4           Virginia (35-3)  Tony Bennett       Texas Tech  85-77 (OT)   

            game_site season_year  
0       Houston, Tex.        2023  
1    New Orleans, La.        2022  
2  Indianapolis, Ind.        2021  
3                  --        2020  
4  Minneapolis, Minn.        2019  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   champions    85 non-null     object
 1   coach        85 non-null     object
 2   runner_up    85 non-null     object
 3   final_score  85 non-null   

In [80]:
# Insert data into postseason table

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# insert the data
for index, row in API_data.iterrows():
     cursor.execute("INSERT INTO postseason (champion,season_year,final_score,game_site,coach,runner_up) VALUES (%s,%s,%s,%s,%s,%s)", (row.champions, row.season_year, row.final_score, row.game_site, row.coach,row.runner_up))

# confirm data added by querying
conn.commit()

try:

        cursor.execute('SELECT * FROM postseason;')

        for row in cursor.fetchall():
                print(row)
        
        cursor.close()

except:
        print ("Error: unable to fetch data")

# close connection
conn.close()

(1, 'UConn (31-8)', 2023, '76-59', 'Houston, Tex.', 'Dan Hurley', 'San Diego State')
(2, 'Kansas (34-6)', 2022, '72-69', 'New Orleans, La.', 'Bill Self', 'North Carolina')
(3, 'Baylor (28-2)', 2021, '86-70', 'Indianapolis, Ind.', 'Scott Drew', 'Gonzaga')
(4, 'Canceled due to Covid-19', 2020, '--', '--', '--', '--')
(5, 'Virginia (35-3)', 2019, '85-77 (OT)', 'Minneapolis, Minn.', 'Tony Bennett', 'Texas Tech')
(6, 'Villanova (36-4)', 2018, '79-62', 'San Antonio, Tex.', 'Jay Wright', 'Michigan')
(7, 'North Carolina (33-7)', 2017, '71-65', 'Phoenix, Ariz.', 'Roy Williams', 'Gonzaga')
(8, 'Villanova (35-5)', 2016, '77-74', 'Houston, Texas', 'Jay Wright', 'North Carolina')
(9, 'Duke (35-4)', 2015, '68-63', 'Indianapolis, Ind.', 'Mike Krzyzewski', 'Wisconsin')
(10, 'Connecticut (32-8)', 2014, '60-54', 'Arlington, Texas', 'Kevin Ollie', 'Kentucky')
(11, 'Louisville (35-5)*', 2013, '82-76', 'Atlanta, Ga.', 'Rick Pitino', 'Michigan')
(12, 'Kentucky (38-2)', 2012, '67-59', 'New Orleans, La.', 'Jo

#### Add foreign keys now that values are added to the tables:

In [81]:
# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# remove foreign key constraints

try:
    cursor.execute('SET GLOBAL FOREIGN_KEY_CHECKS=0;')

except:
    print ("Error: unable to submit query")

# close connection
conn.close()

In [82]:
# insert foreign key for Table 1: team_info

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the foreign key
try:

    cursor.execute("ALTER TABLE team_info ADD game_details_key INT NOT NULL, ADD FOREIGN KEY (game_details_key) REFERENCES game_details(game_details_id);")

except:
    print ("Error: unable to submit query")

# close connection
conn.close()

In [83]:
# insert foreign key for Table 2: postseason

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the foreign key
try:

    cursor.execute("ALTER TABLE postseason ADD team_key INT NOT NULL, ADD FOREIGN KEY (team_key) REFERENCES team_info(team_id);")

except:
    print ("Error: unable to submit query")

# close connection
conn.close()

In [84]:
# insert foreign key for Table 3: skills_offense

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the foreign key
try:

    cursor.execute("ALTER TABLE skills_offense ADD postseason_key INT NOT NULL, ADD FOREIGN KEY (postseason_key) REFERENCES postseason(postseason_id);")

except:
    print ("Error: unable to submit query")

# close connection
conn.close()

In [85]:
# insert foreign key for Table 4: skills_defense

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the foreign key
try:

    cursor.execute("ALTER TABLE skills_defense ADD skills_offense_key INT NOT NULL, ADD FOREIGN KEY (skills_offense_key) REFERENCES skills_offense(skills_offense_id);")

except:
    print ("Error: unable to submit query")

# close connection
conn.close()

In [86]:
# insert foreign key for Table 5: game_details

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# create the foreign key
try:

    cursor.execute("ALTER TABLE game_details ADD skills_defense_key INT NOT NULL, ADD FOREIGN KEY (skills_defense_key) REFERENCES skills_defense(skills_defense_id);")

except:
    print ("Error: unable to submit query")

# close connection
conn.close()

## Part 3: Query information from the SQL tables

#### Select data from 3 SQL tables

In [87]:
# SQL table = postseason

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# select postseason details from 2019

try:

    cursor.execute('SELECT * FROM postseason WHERE postseason.season_year LIKE 2019;')

    for row in cursor.fetchmany(1):
        print(row)

    cursor.close()

except:
    print ("Error: unable to fetch data")

# close connection
conn.close()

(5, 'Virginia (35-3)', 2019, '85-77 (OT)', 'Minneapolis, Minn.', 'Tony Bennett', 'Texas Tech', 0)


In [92]:
# SQL table = skills_offense, skills_defense

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# select offensive and defense rebound rates grouped by each team

try:

    cursor.execute('SELECT team_name, offensive_rebounds, defensive_rebounds, year_season FROM skills_offense, skills_defense WHERE skills_offense.team_name = skills_defense.school_name AND skills_offense.season_year = skills_defense.year_season ORDER BY year_season;')

    for row in cursor.fetchall():
        print(row)
    
    cursor.close()

except:
    print ("Error: unable to fetch data")

# close connection
conn.close()

('Michigan', 32, 29, 2013)
('Louisville', 38, 33, 2013)
('Ohio St.', 32, 28, 2013)
('Duke', 29, 32, 2013)
('Marquette', 38, 33, 2013)
('Florida', 34, 28, 2013)
('Syracuse', 39, 34, 2013)
('Wichita St.', 38, 26, 2013)
('Gonzaga', 38, 30, 2013)
('Saint Louis', 28, 29, 2013)
('VCU', 37, 35, 2013)
('Butler', 36, 26, 2013)
('Memphis', 36, 31, 2013)
('San Diego St.', 32, 28, 2013)
('Illinois', 34, 33, 2013)
('Creighton', 29, 27, 2013)
('Colorado St.', 41, 23, 2013)
('North Carolina', 35, 32, 2013)
('Temple', 31, 31, 2013)
('Iowa St.', 34, 28, 2013)
('Minnesota', 44, 33, 2013)
('California', 32, 31, 2013)
('Mississippi', 34, 32, 2013)
('Harvard', 26, 31, 2013)
('Georgetown', 30, 31, 2013)
('New Mexico', 29, 28, 2013)
('Kansas St.', 38, 34, 2013)
('Wisconsin', 33, 27, 2013)
('Oklahoma St.', 31, 31, 2013)
('UNLV', 33, 27, 2013)
('UCLA', 31, 34, 2013)
('Notre Dame', 34, 30, 2013)
('Pittsburgh', 40, 30, 2013)
('North Carolina St.', 35, 33, 2013)
('Villanova', 33, 30, 2013)
('Missouri', 39, 29, 20

In [89]:
# SQL table = team_info

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# select team with most games won, added together for all seasons listed

try:

    cursor.execute('SELECT school_name AS team_name, SUM(games_won) AS total_games_won FROM team_info GROUP BY team_name ORDER BY total_games_won DESC LIMIT 1;')

    for row in cursor.fetchall():
        print(row)
    
    cursor.close()

except:
    print ("Error: unable to fetch data")

# close connection
conn.close()

('Gonzaga', Decimal('307'))


#### Select using aggregation

In [90]:
# SQL table = skills_offense

# open mySQL connection
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

# select average freethrow rate and group by conference

try:

    cursor.execute('SELECT conference, AVG(freethrow_rate) AS average_freethrow_rate FROM skills_offense GROUP BY skills_offense.conference ORDER BY average_freethrow_rate;')

    for row in cursor.fetchmany(10):
        print(row)
    
    cursor.close()

except:
    print ("Error: unable to fetch data")

# close connection
conn.close()

('Pat', Decimal('32.9000'))
('CUSA', Decimal('33.3000'))
('SC', Decimal('33.5000'))
('AE', Decimal('33.7000'))
('ACC', Decimal('33.8308'))
('B10', Decimal('33.9861'))
('Sum', Decimal('34.5000'))
('WCC', Decimal('34.6316'))
('MVC', Decimal('34.8571'))
('NEC', Decimal('35.0000'))
